# **Chapter 5: Proof of Work (PoW)**

---

## **5.1 How Proof of Work Operates**

### **5.1.1 Mining Process Explained**

**Proof of Work (PoW)** is the original consensus mechanism used by Bitcoin and (initially) Ethereum. It requires miners to perform computationally intensive work to add new blocks to the blockchain. This "work" involves finding a specific hash value that meets certain criteria, making it prohibitively expensive to attack the network but relatively easy to verify.

#### **The Mining Concept**

Imagine a lottery where you win by guessing a number. The only way to win is to try guesses repeatedly until you find the right one. In PoW, miners compete to find a "nonce" (number used once) that, when combined with block data, produces a hash below a certain target.

```
The Mining Lottery Analogy:

┌─────────────────────────────────────────────────────────────┐
│                    THE MINING LOTTERY                       │
│                                                             │
│  Target: Find a hash starting with "0000"                   │
│                                                             │
│  Block Data: "Alice pays Bob 5 BTC + Timestamp + Nonce"    │
│                                                             │
│  Attempt 1:                                                 │
│    Nonce = 1                                                │
│    Hash = 0x7a3f...8b2e (Doesn't start with 0000) ✗        │
│                                                             │
│  Attempt 2:                                                 │
│    Nonce = 2                                                │
│    Hash = 0x2c9d...4f1a (Doesn't start with 0000) ✗        │
│                                                             │
│  Attempt 3:                                                 │
│    Nonce = 3                                                │
│    Hash = 0x0000...a7b3 (Starts with 0000!) ✓              │
│                                                             │
│  WINNER! Block is valid and can be added to chain          │
│                                                             │
│  Key Insight: Finding the hash is hard (requires work),    │
│  but verifying it is easy (just hash once)                 │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

#### **The Mathematical Puzzle**

The core of PoW is the **hash puzzle**:

$$\text{SHA-256}(\text{Block Header} + \text{Nonce}) < \text{Target}$$

Where:
- **Block Header**: Contains previous hash, timestamp, transactions, Merkle root
- **Nonce**: A 32-bit number miners increment to find valid hash
- **Target**: A threshold value determining difficulty (lower = harder)

```
Hash Target Visualization:

┌─────────────────────────────────────────────────────────────┐
│                                                             │
│  Difficulty Target (Easy):                                  │
│  Hash must be less than: 0x0000ffffffffffffffff...          │
│                                                             │
│  Valid Hashes:                                              │
│  0x00001234... ✓ (starts with many zeros)                  │
│  0x0000abcd... ✓                                            │
│                                                             │
│  Invalid Hashes:                                            │
│  0x12345678... ✗ (doesn't start with enough zeros)          │
│                                                             │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Difficulty Target (Hard):                                  │
│  Hash must be less than: 0x00000000ffffffff...              │
│                                                             │
│  Valid Hashes:                                              │
│  0x000000001... ✓ (starts with 8 zeros)                    │
│                                                             │
│  Invalid Hashes:                                            │
│  0x00001234... ✗ (only 4 zeros, not enough)                 │
│                                                             │
│  More leading zeros = Higher difficulty = More work needed  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### **5.1.2 Difficulty Target and Adjustment**

The **difficulty** ensures blocks are mined at consistent intervals (Bitcoin: ~10 minutes, Ethereum pre-merge: ~15 seconds). As more miners join and computing power increases, difficulty adjusts upward to maintain this schedule.

#### **How Difficulty Works**

Difficulty is inversely proportional to the target:
- **Higher Difficulty** = **Lower Target** = **Harder to find valid hash**
- **Lower Difficulty** = **Higher Target** = **Easier to find valid hash**

```
Difficulty Adjustment Visualization:

┌─────────────────────────────────────────────────────────────┐
│  Bitcoin Difficulty Adjustment (Every 2016 blocks ≈ 2 weeks)│
│                                                             │
│  Scenario 1: Blocks too fast (8 minutes average)            │
│  ┌───────────────────────────────────────────────────────┐  │
│  │ Target: 0x00000000ffffffffffffffff...                │  │
│  │                                                       │  │
│  │ New Target: 0x0000000000ffffffffffff... (Smaller)    │  │
│  │ Difficulty: INCREASED by 25%                         │  │
│  │                                                       │  │
│  │ Result: Harder to mine, slows down to 10 min         │  │
│  └───────────────────────────────────────────────────────┘  │
│                                                             │
│  Scenario 2: Blocks too slow (12 minutes average)           │
│  ┌───────────────────────────────────────────────────────┐  │
│  │ Target: 0x00000000ffffffffffffffff...                │  │
│  │                                                       │  │
│  │ New Target: 0x0000ffffffffffffffffff... (Larger)     │  │
│  │ Difficulty: DECREASED by 20%                          │  │
│  │                                                       │  │
│  │ Result: Easier to mine, speeds up to 10 min          │  │
│  └───────────────────────────────────────────────────────┘  │
│                                                             │
│  Formula: New Difficulty = Old Difficulty × (Actual Time / │
│           Target Time)                                       │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Code Example: Difficulty Calculation**

```javascript
// difficulty.js
// Demonstrates how difficulty targets work in PoW

const crypto = require('crypto');

class DifficultyCalculator {
    constructor() {
        // Bitcoin uses a maximum target (easiest possible)
        // Represented as 0x00000000FFFF0000000000000000000000000000000000000000000000000000
        this.maxTarget = BigInt("0x00000000FFFF0000000000000000000000000000000000000000000000000000");
    }

    /**
     * Calculate difficulty from target
     * Difficulty = MaxTarget / CurrentTarget
     */
    calculateDifficulty(target) {
        const targetBigInt = BigInt("0x" + target);
        const difficulty = this.maxTarget / targetBigInt;
        return difficulty.toString();
    }

    /**
     * Calculate target from difficulty
     * Target = MaxTarget / Difficulty
     */
    calculateTarget(difficulty) {
        const target = this.maxTarget / BigInt(difficulty);
        return target.toString(16).padStart(64, '0');
    }

    /**
     * Check if a hash meets the target (is valid)
     */
    isHashValid(hash, target) {
        const hashValue = BigInt("0x" + hash);
        const targetValue = BigInt("0x" + target);
        return hashValue < targetValue;
    }

    /**
     * Simulate difficulty adjustment (Bitcoin style)
     * Adjusts every 2016 blocks to maintain 10-minute average
     */
    adjustDifficulty(currentDifficulty, actualTimeMinutes, targetTimeMinutes = 20160) {
        // targetTimeMinutes = 2016 blocks × 10 minutes = 20160 minutes
        
        console.log('\n=== DIFFICULTY ADJUSTMENT ===');
        console.log(`Current Difficulty: ${currentDifficulty}`);
        console.log(`Actual Time: ${actualTimeMinutes} minutes`);
        console.log(`Target Time: ${targetTimeMinutes} minutes`);
        
        // Calculate adjustment factor
        const adjustment = actualTimeMinutes / targetTimeMinutes;
        console.log(`Adjustment Factor: ${adjustment.toFixed(4)}`);
        
        // Bitcoin limits adjustment to 4x or 0.25x per cycle (anti-manipulation)
        let boundedAdjustment = adjustment;
        if (adjustment > 4) boundedAdjustment = 4;
        if (adjustment < 0.25) boundedAdjustment = 0.25;
        
        if (boundedAdjustment !== adjustment) {
            console.log(`Bounded Adjustment: ${boundedAdjustment} (safety limits applied)`);
        }
        
        const newDifficulty = Math.floor(currentDifficulty * boundedAdjustment);
        console.log(`New Difficulty: ${newDifficulty}`);
        
        if (newDifficulty > currentDifficulty) {
            console.log('📈 Difficulty INCREASED (blocks were too fast)');
        } else if (newDifficulty < currentDifficulty) {
            console.log('📉 Difficulty DECREASED (blocks were too slow)');
        } else {
            console.log('➡️  Difficulty UNCHANGED');
        }
        
        return newDifficulty;
    }

    displayTargetVisual(target) {
        // Visualize the target as binary to show leading zeros
        const binary = BigInt("0x" + target).toString(2).padStart(256, '0');
        const leadingZeros = binary.match(/^0*/)[0].length;
        
        console.log('\nTarget Visualization:');
        console.log(`Hex: 0x${target.substring(0, 16)}...`);
        console.log(`Leading zeros in binary: ${leadingZeros}`);
        console.log(`Binary: ${binary.substring(0, 32)}...`);
        console.log(`Difficulty: ${this.calculateDifficulty(target)}`);
    }
}

// ============= DEMONSTRATION =============

const calc = new DifficultyCalculator();

console.log('=== PROOF OF WORK DIFFICULTY EXPLAINED ===\n');

// Example 1: Easy target
console.log('Example 1: Easy Target (Low Difficulty)');
const easyTarget = "0000ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff";
calc.displayTargetVisual(easyTarget);

// Example 2: Hard target (Bitcoin current difficulty approx)
console.log('\nExample 2: Hard Target (High Difficulty)');
const hardTarget = "0000000000000000000b3f00000000000000000000000000000000000000000000";
calc.displayTargetVisual(hardTarget);

// Example 3: Difficulty adjustment simulation
console.log('\n' + '='.repeat(60));
console.log('DIFFICULTY ADJUSTMENT SIMULATION');
console.log('='.repeat(60));

// Scenario: Blocks mined too fast (only took 1 week instead of 2)
let currentDiff = 10000000000000;
const actualTime = 10080; // 1 week in minutes
calc.adjustDifficulty(currentDiff, actualTime, 20160);

// Scenario: Blocks mined too slow (took 3 weeks)
console.log('\n--- SCENARIO 2: SLOW BLOCKS ---');
const actualTime2 = 30240; // 3 weeks in minutes
calc.adjustDifficulty(currentDiff, actualTime2, 20160);

// Example 4: Validating hashes against target
console.log('\n' + '='.repeat(60));
console.log('HASH VALIDATION');
console.log('='.repeat(60));

const testTarget = "00000fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff";
const hash1 = "00000a7b3c4d5e6f..."; // Valid (starts with 5 zeros)
const hash2 = "0000a7b3c4d5e6f...";  // Invalid (only 4 zeros)

console.log(`Target: 0x${testTarget.substring(0, 16)}...`);
console.log(`Hash 1: 0x${hash1} - Valid? ${calc.isHashValid(hash1 + "0".repeat(57), testTarget)}`);
console.log(`Hash 2: 0x${hash2} - Valid? ${calc.isHashValid(hash2 + "0".repeat(58), testTarget)}`);

/*
EXPECTED OUTPUT EXPLANATION:

1. Shows how difficulty relates to target size
2. Demonstrates Bitcoin's difficulty adjustment algorithm
3. Shows safety bounds (4x max adjustment)
4. Validates that hashes must be numerically less than target

Key Concepts:
- Lower target = Higher difficulty
- Difficulty adjusts to maintain block time
- Hash must be < target to be valid
- Safety limits prevent manipulation
*/
```

**Line-by-Line Explanation:**

1. **`maxTarget`**: The easiest possible target in Bitcoin (difficulty = 1). All other targets are harder (smaller numbers).

2. **`calculateDifficulty(target)`**: Computes how many times harder current mining is compared to minimum difficulty. Formula: $Difficulty = \frac{MaxTarget}{CurrentTarget}$

3. **`calculateTarget(difficulty)`**: Reverse calculation to find what target corresponds to a given difficulty.

4. **`isHashValid(hash, target)`**: Core validation logic. Converts hex strings to BigInt and checks if hash < target numerically.

5. **`adjustDifficulty()`**: Implements Bitcoin's difficulty adjustment algorithm:
   - Measures actual time vs expected time (2016 blocks × 10 min)
   - Adjusts proportionally but bounded (0.25x to 4x) to prevent manipulation
   - If miners are too fast, difficulty increases

6. **`displayTargetVisual()`**: Shows binary representation to visualize why "more leading zeros = harder".

### **5.1.3 Nonce and Hash Searching**

The **nonce** (Number Used Once) is the variable miners change to find a valid hash. It's a 32-bit integer (0 to 4,294,967,295) that miners increment in their search for a hash meeting the target.

```
Mining Process Flow:

┌─────────────────────────────────────────────────────────────┐
│  MINING ALGORITHM                                           │
│                                                             │
│  1. START                                                   │
│     │                                                       │
│     ▼                                                       │
│  2. Prepare Block Header                                    │
│     • Previous block hash                                   │
│     • Merkle root (transaction hash)                        │
│     • Timestamp                                             │
│     • Difficulty target                                     │
│     │                                                       │
│     ▼                                                       │
│  3. Set Nonce = 0                                           │
│     │                                                       │
│     ▼                                                       │
│  4. Calculate Hash = SHA-256(Block Header + Nonce)         │
│     │                                                       │
│     ▼                                                       │
│  5. Is Hash < Target?                                       │
│     │                                                       │
│     ├─ YES ──▶ 6. Block Found! Broadcast to network        │
│     │                                                       │
│     └─ NO ───▶ 7. Increment Nonce                          │
│                │                                            │
│                ▼                                            │
│             8. Nonce Overflow?                              │
│                │                                            │
│                ├─ YES ──▶ 9. Update Timestamp/Merkle Root  │
│                │            (or add extraNonce in coinbase)  │
│                │            Return to step 3                 │
│                │                                            │
│                └─ NO ───▶ Return to step 4                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Code Example: Complete Mining Simulation**

```javascript
// mining-simulator.js
// Complete Proof of Work mining simulation

const crypto = require('crypto');

class Block {
    constructor(index, transactions, previousHash, timestamp = Date.now()) {
        this.index = index;
        this.timestamp = timestamp;
        this.transactions = transactions;
        this.previousHash = previousHash;
        this.nonce = 0;
        this.difficulty = 4; // Number of leading zeros required (simplified)
        this.hash = this.mineBlock(); // Mine on creation
    }

    /**
     * Calculate SHA-256 hash of block data
     */
    calculateHash() {
        const data = this.index + 
                    this.timestamp + 
                    JSON.stringify(this.transactions) + 
                    this.previousHash + 
                    this.nonce;
        
        return crypto.createHash('sha256').update(data).digest('hex');
    }

    /**
     * MINING FUNCTION
     * Proof of Work implementation
     */
    mineBlock() {
        console.log(`\n⛏️  Mining Block ${this.index}...`);
        console.log(`Difficulty: ${this.difficulty} leading zeros required`);
        
        const target = "0".repeat(this.difficulty); // e.g., "0000"
        let hash = this.calculateHash();
        let attempts = 0;
        const startTime = Date.now();

        // Keep incrementing nonce until hash meets difficulty target
        while (hash.substring(0, this.difficulty) !== target) {
            this.nonce++;
            hash = this.calculateHash();
            attempts++;
            
            // Log progress every 100,000 attempts for visibility
            if (attempts % 100000 === 0) {
                process.stdout.write(`\rAttempts: ${attempts.toLocaleString()}, Current hash: ${hash.substring(0, 16)}...`);
            }

            // Safety check for demonstration (prevent infinite loop if difficulty too high)
            if (attempts > 5000000) {
                console.log('\n⚠️  Too many attempts for demo, lowering difficulty...');
                this.difficulty--;
                return this.mineBlock(); // Retry with lower difficulty
            }
        }

        const duration = Date.now() - startTime;
        
        console.log(`\n✅ Block Mined!`);
        console.log(`Hash: ${hash}`);
        console.log(`Nonce: ${this.nonce.toLocaleString()}`);
        console.log(`Attempts: ${attempts.toLocaleString()}`);
        console.log(`Time: ${duration}ms (${(duration/1000).toFixed(2)}s)`);
        console.log(`Hash Rate: ${(attempts / (duration / 1000)).toFixed(0)} H/s`);
        
        return hash;
    }

    display() {
        console.log(`\n┌──────────────────────────────────────────────┐`);
        console.log(`│ Block ${this.index.toString().padEnd(39)}│`);
        console.log(`├──────────────────────────────────────────────┤`);
        console.log(`│ Hash:        ${this.hash.substring(0, 20)}... │`);
        console.log(`│ Previous:     ${this.previousHash.substring(0, 20)}... │`);
        console.log(`│ Nonce:        ${this.nonce.toString().padEnd(31)}│`);
        console.log(`│ Difficulty:   ${this.difficulty.toString().padEnd(31)}│`);
        console.log(`│ Timestamp:    ${this.timestamp.toString().padEnd(31)}│`);
        console.log(`│ Transactions: ${JSON.stringify(this.transactions).substring(0, 25).padEnd(31)}│`);
        console.log(`└──────────────────────────────────────────────┘`);
    }
}

class Blockchain {
    constructor() {
        this.chain = [this.createGenesisBlock()];
        this.difficulty = 4; // Adjustable difficulty
    }

    createGenesisBlock() {
        // Genesis block has previous hash of "0"
        return new Block(0, "Genesis Block", "0");
    }

    getLatestBlock() {
        return this.chain[this.chain.length - 1];
    }

    addBlock(newBlock) {
        // Set previous hash from latest block
        newBlock.previousHash = this.getLatestBlock().hash;
        
        // Adjust difficulty based on chain length (simulation)
        newBlock.difficulty = this.calculateDifficulty();
        
        // Mine the block
        newBlock.hash = newBlock.mineBlock();
        
        // Add to chain
        this.chain.push(newBlock);
    }

    calculateDifficulty() {
        // Simple difficulty adjustment simulation
        // In real Bitcoin, this adjusts every 2016 blocks
        const baseDifficulty = 4;
        const adjustment = Math.floor(this.chain.length / 2); // Increase every 2 blocks for demo
        return Math.min(baseDifficulty + adjustment, 6); // Cap at 6 for demo speed
    }

    isChainValid() {
        for (let i = 1; i < this.chain.length; i++) {
            const current = this.chain[i];
            const previous = this.chain[i - 1];

            // Verify current block hash
            if (current.hash !== current.calculateHash()) {
                console.log(`Invalid hash at block ${i}`);
                return false;
            }

            // Verify chain linkage
            if (current.previousHash !== previous.hash) {
                console.log(`Broken chain at block ${i}`);
                return false;
            }

            // Verify proof of work
            const target = "0".repeat(current.difficulty);
            if (!current.hash.startsWith(target)) {
                console.log(`Invalid proof of work at block ${i}`);
                return false;
            }
        }
        return true;
    }
}

// ============= DEMONSTRATION =============

console.log('=== PROOF OF WORK BLOCKCHAIN SIMULATION ===\n');

const myBlockchain = new Blockchain();

console.log('Mining Block 1...');
myBlockchain.addBlock(new Block(1, ["Alice pays Bob 5 BTC"], myBlockchain.getLatestBlock().hash));

console.log('\nMining Block 2...');
myBlockchain.addBlock(new Block(2, ["Bob pays Charlie 2 BTC"], myBlockchain.getLatestBlock().hash));

console.log('\nMining Block 3... (Difficulty increased!)');
myBlockchain.addBlock(new Block(3, ["Charlie pays David 1 BTC"], myBlockchain.getLatestBlock().hash));

console.log('\n' + '='.repeat(60));
console.log('BLOCKCHAIN SUMMARY');
console.log('='.repeat(60));

myBlockchain.chain.forEach(block => block.display());

console.log('\n' + '='.repeat(60));
console.log(`Chain Valid: ${myBlockchain.isChainValid()}`);
console.log('='.repeat(60));

// Demonstrate tampering detection
console.log('\n=== SECURITY DEMONSTRATION ===');
console.log('Attempting to tamper with Block 1...');
myBlockchain.chain[1].transactions = ["Alice pays Bob 100 BTC"]; // Changed amount

console.log(`Chain Valid after tampering: ${myBlockchain.isChainValid()}`);
console.log('❌ Tampering detected because hash no longer matches!');

/*
EXPECTED OUTPUT EXPLANATION:

1. Shows live mining with difficulty target
2. Displays nonce value required to find valid hash
3. Shows hash rate (computations per second)
4. Demonstrates increasing difficulty
5. Validates chain integrity (tampering detection)

Key Learning:
- Mining requires brute force search (incrementing nonce)
- Higher difficulty = more leading zeros = exponentially harder
- Changing any data invalidates the proof of work
- Verification is instant (single hash calculation)
*/
```

**Key Implementation Details:**

1. **`mineBlock()`**: The core PoW algorithm. It brute-forces nonce values until finding a hash with required leading zeros.

2. **`difficulty`**: Controls how many leading zeros are required. Each additional zero makes mining exponentially harder (16x harder per zero in hex).

3. **`calculateHash()`**: Deterministic function—same input always produces same output. Changing nonce changes hash unpredictably.

4. **`isChainValid()`**: Verifies the chain by recalculating hashes. Any tampering breaks the proof of work.

5. **Performance Metrics**: Tracks attempts, time, and hash rate to show mining economics.

---

## **5.2 Security Model**

### **5.2.1 51% Attack Explained**

The **51% attack** is the primary security threat to PoW blockchains. If a single entity controls more than 50% of the network's hashing power, they can potentially rewrite transaction history.

#### **How a 51% Attack Works**

```
51% Attack Scenario:

┌─────────────────────────────────────────────────────────────┐
│  NORMAL NETWORK (Honest miners control 100%)               │
│                                                             │
│  Height 1 → Height 2 → Height 3 → Height 4 → Height 5    │
│     │          │          │          │          │           │
│   Honest     Honest    Honest    Honest    Honest         │
│                                                             │
│  ✓ All miners agree on longest chain                      │
│  ✓ Transactions immutable after confirmations               │
│                                                             │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  ATTACK BEGINS (Attacker gains 51% hash power)             │
│                                                             │
│  Public Chain (49% power):                                 │
│  Height 1 → Height 2 → Height 3 → Height 4                 │
│                                        │                    │
│                                        ▼                    │
│  Attacker's Secret Chain (51% power):                      │
│  Height 1 → Height 2 → Height 3 → Height 4' → Height 5'    │
│                                        ↑                    │
│                                    Fork point               │
│                                                             │
│  Attacker mines privately, faster than public chain        │
│  (because they have more hash power)                        │
│                                                             │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  DOUBLE SPEND EXECUTION                                    │
│                                                             │
│  Step 1: Attacker spends coins on public chain              │
│          (e.g., deposits to exchange at Height 4)          │
│                                                             │
│  Step 2: Attacker continues mining secretly                 │
│          (reaches Height 6' while public at Height 4)      │
│                                                             │
│  Step 3: Attacker releases secret chain                     │
│                                                             │
│  Public sees: Height 1 → 2 → 3 → 4' → 5' → 6' (LONGER)   │
│                                                             │
│  Result:                                                    │
│  ✓ Public chain (with deposit) is orphaned                │
│  ✓ Attacker's chain (without deposit) becomes canonical     │
│  ✓ Attacker keeps coins AND received goods from deposit     │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Code Example: Simulating Attack Resistance**

```javascript
// attack-simulation.js
// Simulates 51% attack probability and economics

class AttackSimulator {
    constructor() {
        this.blockTime = 600; // 10 minutes in seconds (Bitcoin)
    }

    /**
     * Calculate probability of attacker succeeding
     * Based on the Bitcoin whitepaper formula
     */
    calculateAttackProbability(attackerHashRate, confirmations) {
        // q = attacker hash rate proportion
        // p = honest network hash rate proportion (1 - q)
        const q = attackerHashRate; // e.g., 0.51 for 51%
        const p = 1 - q;
        
        // If attacker has <= 50%, probability approaches 0 over time
        // If attacker has > 50%, probability approaches 1
        
        if (q <= 0.5) {
            // Approximation for q < 0.5
            // Probability decreases exponentially with confirmations
            const lambda = confirmations * (1 - (q / p));
            return Math.exp(-lambda);
        } else {
            // Attacker with > 50% will eventually win
            return 1.0;
        }
    }

    /**
     * Calculate cost of 51% attack
     */
    calculateAttackCost(networkHashRate, hardwareCostPerTh, electricityCost) {
        // networkHashRate in TH/s (Terahashes per second)
        // hardwareCostPerTh in USD
        // electricityCost per day
        
        const attackerHashRate = networkHashRate; // Need to match network
        const hardwareInvestment = attackerHashRate * hardwareCostPerTh;
        const dailyElectricity = electricityCost;
        
        console.log('\n=== 51% ATTACK ECONOMICS ===');
        console.log(`Network Hash Rate: ${networkHashRate.toLocaleString()} TH/s`);
        console.log(`Required Attacker Hash Rate: ${attackerHashRate.toLocaleString()} TH/s (51%)`);
        console.log(`Hardware Investment: $${hardwareInvestment.toLocaleString()}`);
        console.log(`Daily Electricity: $${dailyElectricity.toLocaleString()}`);
        
        return {
            hardwareCost: hardwareInvestment,
            dailyOperationalCost: dailyElectricity
        };
    }

    /**
     * Simulate attack success probability over time
     */
    simulateAttack(attackerPower, targetConfirmations) {
        console.log(`\n=== ATTACK SIMULATION ===`);
        console.log(`Attacker Power: ${(attackerPower * 100).toFixed(1)}%`);
        console.log(`Target Confirmations: ${targetConfirmations}`);
        
        if (attackerPower <= 0.5) {
            const prob = this.calculateAttackProbability(attackerPower, targetConfirmations);
            console.log(`Success Probability: ${((1 - prob) * 100).toFixed(6)}%`);
            console.log(`Failure Probability: ${(prob * 100).toFixed(6)}%`);
            
            // Calculate expected time to success
            const blocksNeeded = targetConfirmations;
            const attackerAdvantage = (0.5 - attackerPower) / 0.5; // How far from 51%
            const expectedBlocks = blocksNeeded / attackerAdvantage;
            const expectedTime = (expectedBlocks * this.blockTime) / 3600; // hours
            
            console.log(`Expected time to catch up: ${expectedTime.toFixed(2)} hours`);
        } else {
            console.log('Success Probability: ~100% (given enough time)');
            console.log('Attacker will eventually outpace honest chain');
        }
    }

    /**
     * Calculate security budget (miner's incentive to stay honest)
     */
    calculateSecurityBudget(blockReward, txFees, btcPrice, dailyBlocks = 144) {
        const dailyReward = blockReward * dailyBlocks * btcPrice;
        const dailyFees = txFees * dailyBlocks;
        const totalDaily = dailyReward + dailyFees;
        
        console.log('\n=== NETWORK SECURITY BUDGET ===');
        console.log(`Daily Block Rewards: $${dailyReward.toLocaleString()}`);
        console.log(`Daily Transaction Fees: $${dailyFees.toLocaleString()}`);
        console.log(`Total Daily Security Budget: $${totalDaily.toLocaleString()}`);
        console.log(`Annual Security Budget: $${(totalDaily * 365).toLocaleString()}`);
        
        return totalDaily;
    }
}

// ============= DEMONSTRATION =============

const simulator = new AttackSimulator();

console.log('=== PROOF OF WORK SECURITY ANALYSIS ===\n');

// Scenario 1: Small attacker (30%)
console.log('SCENARIO 1: Attacker with 30% hash power');
simulator.simulateAttack(0.30, 6);

// Scenario 2: Large attacker (49%)
console.log('\nSCENARIO 2: Attacker with 49% hash power');
simulator.simulateAttack(0.49, 6);

// Scenario 3: Majority attacker (51%)
console.log('\nSCENARIO 3: Attacker with 51% hash power');
simulator.simulateAttack(0.51, 6);

// Bitcoin network economics (approximate values)
console.log('\n' + '='.repeat(60));
console.log('BITCOIN NETWORK ECONOMICS (Approximate)');
console.log('='.repeat(60));

// Approximate Bitcoin network stats
const networkHashRate = 500000000; // 500 EH/s = 500,000,000 TH/s
const hardwareCost = 50; // $50 per TH/s (ASIC miners)
const dailyElectricity = 10000000; // $10M per day (estimated)

simulator.calculateAttackCost(networkHashRate, hardwareCost, dailyElectricity);
simulator.calculateSecurityBudget(6.25, 0.5, 40000); // 6.25 BTC reward, $40k price

console.log('\n' + '='.repeat(60));
console.log('SECURITY ANALYSIS');
console.log('='.repeat(60));
console.log('1. Cost to attack Bitcoin: Billions in hardware + millions daily');
console.log('2. More confirmations = exponentially harder to reverse');
console.log('3. Economic incentive: Cheaper to mine honestly than attack');
console.log('4. Game theory: Attack would destroy BTC value, hurting attacker');

/*
EXPECTED OUTPUT EXPLANATION:

Shows that:
- 30% attacker has near-zero chance of success
- 49% attacker has very low probability (decreases with confirmations)
- 51% attacker will eventually succeed but costs billions
- Security comes from economic cost, not just cryptography
*/
```

### **5.2.2 Energy Consumption and Environmental Concerns**

PoW requires significant computational effort, leading to high energy consumption. This is a feature (security through cost) but also a criticism.

```
Energy Consumption Analysis:

┌─────────────────────────────────────────────────────────────┐
│  BITCOIN ENERGY CONSUMPTION (Estimated)                    │
│                                                             │
│  Annual Consumption: ~150 TWh/year                         │
│  Comparable to: Argentina or Netherlands                     │
│                                                             │
│  Security Justification:                                    │
│  ┌───────────────────────────────────────────────────────┐ │
│  │ Energy isn't "wasted" - it's purchasing security       │ │
│  │                                                        │ │
│  │ Cost to attack = Cost of energy to mine alternative    │ │
│  │ chain                                                  │ │
│  │                                                        │ │
│  │ Higher energy = Higher security budget                 │ │
│  │                                                        │ │
│  │ Analogous to: Gold mining, military defense spending  │ │
│  └───────────────────────────────────────────────────────┘ │
│                                                             │
│  Environmental Mitigation:                                    │
│  • Renewable energy usage (estimated 50%+ of Bitcoin mining) │
│  • Stranded energy utilization (flared natural gas)        │
│  • Grid stabilization services (demand response)          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### **5.2.3 Strengths and Weaknesses**

```
┌─────────────────────────────────────────────────────────────────┐
│              PROOF OF WORK: STRENGTHS                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ✓ PROVEN SECURITY                                             │
│    • 15+ years of Bitcoin operation without major breaches     │
│    • Economic cost makes attacks prohibitively expensive       │
│                                                                 │
│  ✓ DECENTRALIZATION                                             │
│    • Anyone can participate (permissionless)                   │
│    • No coin ownership required to validate                    │
│    • Difficult to coordinate censorship                        │
│                                                                 │
│  ✓ FAIR DISTRIBUTION                                            │
│    • Coins distributed through work, not pre-mined             │
│    • Meritocratic (more work = more reward)                   │
│                                                                 │
│  ✓ ROBUST CONSENSUS                                             │
│    • Clear objective rules (longest chain wins)               │
│    • Forks resolve automatically                                │
│    • No central coordinator needed                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│              PROOF OF WORK: WEAKNESSES                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ✗ ENERGY CONSUMPTION                                           │
│    • High electricity usage                                     │
│    • Environmental concerns                                     │
│    • Regulatory scrutiny                                        │
│                                                                 │
│  ✗ SCALABILITY LIMITS                                          │
│    • Slow block times (10 min Bitcoin, 15 sec Ethereum)       │
│    • Limited throughput (~7 TPS Bitcoin)                        │
│    • High fees during congestion                                │
│                                                                 │
│  ✗ HARDWARE CENTRALIZATION                                      │
│    • ASICs create barrier to entry                              │
│    • Mining pools concentrate power                             │
│    • Geographical centralization (cheap electricity regions)   │
│                                                                 │
│  ✗ 51% ATTACK VULNERABILITY                                    │
│    • Theoretical possibility (though expensive)                  │
│    • Smaller networks vulnerable                                │
│    • Requires constant vigilance                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **5.3 Mining Hardware Evolution**

The hardware used for mining has evolved dramatically, from general-purpose CPUs to specialized ASICs.

### **5.3.1 CPU Mining**

```
CPU Mining Era (2009-2010):

┌─────────────────────────────────────────────────────────────┐
│  Hardware: Standard Computer Processors                       │
│  Efficiency: ~1-10 MH/s per CPU                               │
│  Cost: Very low (used existing hardware)                    │
│                                                             │
│  ┌───────────────────────────────────────────────────────┐  │
│  │  CPU Architecture                                     │  │
│  │                                                       │  │
│  │  ┌─────────┐    ┌─────────┐    ┌─────────┐         │  │
│  │  │ Core 1  │    │ Core 2  │    │ Core 3  │         │  │
│  │  │ SHA-256 │    │ SHA-256 │    │ SHA-256 │         │  │
│  │  └─────────┘    └─────────┘    └─────────┘         │  │
│  │       │              │              │               │  │
│  │       └──────────────┼──────────────┘               │  │
│  │                      │                               │  │
│  │                 ┌────┴────┐                          │  │
│  │                 │ Shared  │                          │  │
│  │                 │ Memory  │                          │  │
│  │                 └─────────┘                          │  │
│  │                                                       │  │
│  │  Problem: General purpose, not optimized for hashing  │  │
│  └───────────────────────────────────────────────────────┘  │
│                                                             │
│  Today: Completely obsolete for Bitcoin                     │
│  Still used for some CPU-only coins (Monero)               │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### **5.3.2 GPU Mining**

```
GPU Mining Era (2010-2013):

┌─────────────────────────────────────────────────────────────┐
│  Hardware: Graphics Processing Units                        │
│  Efficiency: ~100 MH/s - 1 GH/s per GPU                     │
│  Cost: Moderate ($300-$1000 per GPU)                        │
│                                                             │
│  Why GPUs?                                                  │
│  • Massively parallel architecture (thousands of cores)    │
│  • Optimized for repetitive calculations (SHA-256)         │
│  • Can calculate many hashes simultaneously                │
│                                                             │
│  ┌───────────────────────────────────────────────────────┐  │
│  │  GPU Architecture                                     │  │
│  │                                                       │  │
│  │   ┌────┐ ┌────┐ ┌────┐ ┌────┐ ┌────┐ ┌────┐       │  │
│  │   │Core│ │Core│ │Core│ │Core│ │Core│ │Core│ ...    │  │
│  │   │ 1  │ │ 2  │ │ 3  │ │ 4  │ │ 5  │ │ 6  │       │  │
│  │   └────┘ └────┘ └────┘ └────┘ └────┘ └────┘       │  │
│  │     │      │      │      │      │      │           │  │
│  │   ┌──────────────────────────────────────────┐     │  │
│  │   │      Parallel SHA-256 Computation       │     │  │
│  │   │  Each core calculates different nonce    │     │  │
│  │   └──────────────────────────────────────────┘     │  │
│  │                                                       │  │
│  │  Advantage: 100x faster than CPU                  │  │
│  │  Disadvantage: High power consumption, heat        │  │
│  └───────────────────────────────────────────────────────┘  │
│                                                             │
│  Today: Obsolete for Bitcoin, used for Ethereum (pre-PoS)  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### **5.3.3 ASIC Mining**

```
ASIC Mining Era (2013-Present):

┌─────────────────────────────────────────────────────────────┐
│  Hardware: Application-Specific Integrated Circuits         │
│  Efficiency: ~1-100 TH/s per device                         │
│  Cost: $2000-$10000+ per unit                               │
│                                                             │
│  ASIC = Chip designed for ONE specific task                │
│  In this case: SHA-256 hashing only                         │
│                                                             │
│  ┌───────────────────────────────────────────────────────┐  │
│  │  ASIC Architecture                                    │  │
│  │                                                       │  │
│  │  ┌───────────────────────────────────────────────┐   │  │
│  │  │  SHA-256 Hashing Units (Thousands on one chip)│   │  │
│  │  │                                               │   │  │
│  │  │  ┌────────┐ ┌────────┐ ┌────────┐          │   │  │
│  │  │  │ Hash 1 │ │ Hash 2 │ │ Hash 3 │ ...       │   │  │
│  │  │  │ Unit   │ │ Unit   │ │ Unit   │            │   │  │
│  │  │  └────────┘ └────────┘ └────────┘            │   │  │
│  │  │                                               │   │  │
│  │  │  Optimized at silicon level for:            │   │  │
│  │  │  • SHA-256 only                              │   │  │
│  │  │  • Minimal power consumption                 │   │  │
│  │  │  • Maximum hash rate                         │   │  │
│  │  │                                               │   │  │
│  │  │  Cannot be repurposed for other algorithms  │   │  │
│  │  └───────────────────────────────────────────────┘   │  │
│  │                                                       │  │
│  │  Efficiency: 100,000x+ faster than CPU               │  │
│  │  Power: Optimized for performance/watt              │  │
│  └───────────────────────────────────────────────────────┘  │
│                                                             │
│  Impact:                                                    │
│  • Made CPU/GPU mining obsolete for Bitcoin                │
│  • Centralized mining in industrial data centers           │
│  • Created barrier to entry (expensive hardware)            │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Hardware Evolution Summary:**

| Era | Hardware | Hash Rate | Efficiency | Accessibility |
|-----|----------|-----------|------------|---------------|
| 2009-2010 | CPU | 1-10 MH/s | Low | Very High |
| 2010-2013 | GPU | 100 MH/s - 1 GH/s | Medium | High |
| 2013-2015 | FPGA | 1-100 GH/s | High | Medium |
| 2015-Present | ASIC | 1-100 TH/s | Very High | Low |

---

## **5.4 Mining Pools**

### **5.4.1 Centralized vs. Decentralized Pools**

Individual miners face high variance—it might take years for a solo miner to find a block. **Mining pools** combine hash power to share rewards.

```
Mining Pool Architecture:

┌─────────────────────────────────────────────────────────────┐
│                     MINING POOL                            │
│                                                             │
│  Individual Miners                    Pool Coordinator       │
│                                                             │
│  ┌─────────┐  ┌─────────┐  ┌─────────┐      ┌─────────┐ │
│  │ Miner 1 │  │ Miner 2 │  │ Miner 3 │      │  Pool   │ │
│  │  (ASIC) │  │  (ASIC) │  │  (ASIC) │      │ Server  │ │
│  │  10 TH/s│  │  15 TH/s│  │  25 TH/s│      │         │ │
│  └────┬────┘  └────┬────┘  └────┬────┘      └────┬────┘ │
│       │            │            │                │        │
│       └────────────┼────────────┘                │        │
│                    │                            │        │
│                    ▼                            ▼        │
│            ┌──────────────┐            ┌──────────────┐   │
│            │  Combined    │            │ Distributes │   │
│            │  Hash Power  │            │   Rewards   │   │
│            │  50 TH/s     │            │   Pro-rata  │   │
│            └──────────────┘            └──────────────┘   │
│                                                             │
│  Without Pool:                                              │
│  • Miner 1 finds block every ~2 years (solo)               │
│  • High variance, unpredictable income                     │
│                                                             │
│  With Pool:                                                 │
│  • Pool finds blocks regularly (every few hours)           │
│  • Rewards distributed based on shares contributed         │
│  • Steady, predictable income                               │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### **5.4.2 Reward Distribution Methods**

**1. PPS (Pay Per Share)**
- Pool pays fixed amount for each valid share submitted
- Risk transferred to pool operator
- Lower reward (fee taken for risk)

**2. PPLNS (Pay Per Last N Shares)**
- Rewards distributed based on shares in last N rounds
- Higher variance but higher average return
- No risk to pool operator

**3. PROP (Proportional)**
- Rewards proportional to shares in current round
- Simple but vulnerable to pool hopping

```
Reward Distribution Comparison:

┌─────────────────────────────────────────────────────────────┐
│  SCENARIO: Pool finds block (6.25 BTC reward)               │
│                                                             │
│  Miner 1: Contributed 10% of shares                        │
│  Miner 2: Contributed 20% of shares                        │
│  Miner 3: Contributed 70% of shares (Pool operator)        │
│                                                             │
│  PPS (Pay Per Share):                                       │
│  • Each share = fixed amount (e.g., 0.00001 BTC)          │
│  • Paid immediately, regardless of block find                │
│  • Pool takes risk if unlucky                              │
│                                                             │
│  PPLNS (Pay Per Last N Shares):                             │
│  • Miner 1: 0.625 BTC (10%)                                 │
│  • Miner 2: 1.25 BTC (20%)                                  │
│  • Miner 3: 4.375 BTC (70%)                                 │
│  • Paid only when block found                               │
│                                                             │
│  PROP (Proportional):                                       │
│  • Similar to PPLNS but only counts current round          │
│  • Vulnerable to timing attacks                            │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## **5.5 Code Implementation: Simple PoW Blockchain**

Let's build a complete, working Proof of Stake blockchain with difficulty adjustment.

```javascript
// pow-blockchain.js
// Complete Proof of Work Blockchain Implementation

const crypto = require('crypto');

class Transaction {
    constructor(fromAddress, toAddress, amount) {
        this.fromAddress = fromAddress;
        this.toAddress = toAddress;
        this.amount = amount;
        this.timestamp = Date.now();
    }

    calculateHash() {
        const data = this.fromAddress + this.toAddress + this.amount + this.timestamp;
        return crypto.createHash('sha256').update(data).digest('hex');
    }
}

class Block {
    constructor(timestamp, transactions, previousHash = '') {
        this.timestamp = timestamp;
        this.transactions = transactions; // Array of Transaction objects
        this.previousHash = previousHash;
        this.nonce = 0;
        this.difficulty = 2; // Starting difficulty
        this.hash = this.mineBlock();
    }

    /**
     * Calculate hash of block contents
     */
    calculateHash() {
        return crypto.createHash('sha256')
            .update(
                this.previousHash + 
                this.timestamp + 
                JSON.stringify(this.transactions) + 
                this.nonce
            )
            .digest('hex');
    }

    /**
     * Proof of Work Mining
     * Finds nonce such that hash starts with difficulty number of zeros
     */
    mineBlock() {
        const target = "0".repeat(this.difficulty);
        console.log(`⛏️  Mining block with difficulty ${this.difficulty}...`);
        
        const startTime = Date.now();
        
        while (true) {
            const hash = this.calculateHash();
            
            if (hash.substring(0, this.difficulty) === target) {
                const duration = Date.now() - startTime;
                console.log(`✅ Block mined: ${hash}`);
                console.log(`   Nonce: ${this.nonce}`);
                console.log(`   Time: ${duration}ms`);
                return hash;
            }
            
            this.nonce++;
            
            // Safety check for demo
            if (this.nonce > 10000000) {
                throw new Error("Mining taking too long for demo");
            }
        }
    }

    /**
     * Validate that block has valid proof of work
     */
    hasValidProofOfWork() {
        const target = "0".repeat(this.difficulty);
        const hash = this.calculateHash();
        return hash.startsWith(target);
    }

    display() {
        console.log(`\n┌─────────────────────────────────────────────────┐`);
        console.log(`│ Block Hash: ${this.hash.substring(0, 40)}... │`);
        console.log(`│ Previous:   ${this.previousHash.substring(0, 40)}... │`);
        console.log(`│ Timestamp:  ${this.timestamp}                      │`);
        console.log(`│ Nonce:     ${this.nonce.toString().padEnd(39)} │`);
        console.log(`│ Difficulty: ${this.difficulty.toString().padEnd(38)} │`);
        console.log(`│ Transactions: ${this.transactions.length.toString().padEnd(36)} │`);
        this.transactions.forEach((tx, i) => {
            console.log(`│   ${i + 1}. ${tx.fromAddress.substring(0, 8)}... → ${tx.toAddress.substring(0, 8)}... : ${tx.amount} BTC │`);
        });
        console.log(`└─────────────────────────────────────────────────┘`);
    }
}

class Blockchain {
    constructor() {
        this.chain = [this.createGenesisBlock()];
        this.difficulty = 2; // Starting difficulty
        this.pendingTransactions = [];
        this.miningReward = 100; // Reward for mining a block
        this.blockTime = 3000; // Target: 3 seconds per block (for demo)
        this.adjustmentInterval = 5; // Adjust every 5 blocks
    }

    createGenesisBlock() {
        const genesis = new Block(Date.now(), [new Transaction("System", "Genesis", 0)], "0");
        genesis.difficulty = 1; // Genesis is easy
        genesis.hash = genesis.mineBlock();
        return genesis;
    }

    getLatestBlock() {
        return this.chain[this.chain.length - 1];
    }

    /**
     * Adjust difficulty based on actual vs target block time
     */
    adjustDifficulty() {
        if (this.chain.length % this.adjustmentInterval !== 0) {
            return this.difficulty; // Not time to adjust
        }

        // Get block from adjustmentInterval blocks ago
        const lastAdjustmentBlock = this.chain[this.chain.length - this.adjustmentInterval];
        const timeTaken = this.getLatestBlock().timestamp - lastAdjustmentBlock.timestamp;
        const expectedTime = this.blockTime * this.adjustmentInterval;

        console.log(`\n📊 Difficulty Adjustment:`);
        console.log(`   Actual time: ${timeTaken}ms`);
        console.log(`   Expected time: ${expectedTime}ms`);

        if (timeTaken < expectedTime / 2) {
            // Too fast, increase difficulty
            this.difficulty++;
            console.log(`   📈 Difficulty increased to ${this.difficulty}`);
        } else if (timeTaken > expectedTime * 2) {
            // Too slow, decrease difficulty (but not below 1)
            if (this.difficulty > 1) {
                this.difficulty--;
                console.log(`   📉 Difficulty decreased to ${this.difficulty}`);
            }
        } else {
            console.log(`   ➡️  Difficulty unchanged at ${this.difficulty}`);
        }

        return this.difficulty;
    }

    /**
     * Add transaction to pending pool
     */
    addTransaction(transaction) {
        // Verify transaction (simplified)
        if (!transaction.fromAddress || !transaction.toAddress) {
            throw new Error("Transaction must include from and to address");
        }
        
        if (transaction.amount <= 0) {
            throw new Error("Transaction amount must be positive");
        }

        this.pendingTransactions.push(transaction);
        console.log(`📥 Transaction added: ${transaction.fromAddress.substring(0, 8)}... → ${transaction.toAddress.substring(0, 8)}... (${transaction.amount} BTC)`);
    }

    /**
     * Mine pending transactions into a new block
     */
    minePendingTransactions(miningRewardAddress) {
        // Create reward transaction
        const rewardTx = new Transaction("Network", miningRewardAddress, this.miningReward);
        this.pendingTransactions.push(rewardTx);

        // Adjust difficulty if needed
        const blockDifficulty = this.adjustDifficulty();

        // Create new block
        const block = new Block(
            Date.now(),
            this.pendingTransactions,
            this.getLatestBlock().hash
        );
        
        block.difficulty = blockDifficulty;

        // Add to chain
        this.chain.push(block);

        // Clear pending transactions
        this.pendingTransactions = [];

        console.log(`\n💰 Block successfully mined! Miner ${miningRewardAddress.substring(0, 8)}... earned ${this.miningReward} BTC`);
        
        return block;
    }

    /**
     * Calculate balance of address (simplified UTXO model)
     */
    getBalanceOfAddress(address) {
        let balance = 0;

        for (const block of this.chain) {
            for (const trans of block.transactions) {
                if (trans.fromAddress === address) {
                    balance -= trans.amount;
                }
                if (trans.toAddress === address) {
                    balance += trans.amount;
                }
            }
        }

        return balance;
    }

    /**
     * Validate entire chain integrity
     */
    isChainValid() {
        // Check genesis block
        if (JSON.stringify(this.chain[0]) !== JSON.stringify(this.createGenesisBlock())) {
            return false;
        }

        for (let i = 1; i < this.chain.length; i++) {
            const currentBlock = this.chain[i];
            const previousBlock = this.chain[i - 1];

            // Validate proof of work
            if (!currentBlock.hasValidProofOfWork()) {
                console.log(`Invalid proof of work at block ${i}`);
                return false;
            }

            // Check hash integrity
            if (currentBlock.hash !== currentBlock.calculateHash()) {
                console.log(`Invalid hash at block ${i}`);
                return false;
            }

            // Check chain linkage
            if (currentBlock.previousHash !== previousBlock.hash) {
                console.log(`Broken chain at block ${i}`);
                return false;
            }
        }

        return true;
    }

    display() {
        console.log('\n' + '='.repeat(70));
        console.log('BLOCKCHAIN STATE');
        console.log('='.repeat(70));
        
        this.chain.forEach((block, index) => {
            console.log(`\n--- Block ${index} ---`);
            block.display();
        });

        console.log('\n' + '='.repeat(70));
        console.log(`Chain Length: ${this.chain.length}`);
        console.log(`Current Difficulty: ${this.difficulty}`);
        console.log(`Pending Transactions: ${this.pendingTransactions.length}`);
        console.log(`Chain Valid: ${this.isChainValid()}`);
        console.log('='.repeat(70));
    }
}

// ============= DEMONSTRATION =============

console.log('=== PROOF OF WORK BLOCKCHAIN IMPLEMENTATION ===\n');

// Create blockchain
const myCoin = new Blockchain();

// Create some transactions
console.log('Creating transactions...');
myCoin.addTransaction(new Transaction("Alice", "Bob", 50));
myCoin.addTransaction(new Transaction("Bob", "Charlie", 25));

// Mine block 1
console.log('\n--- Mining Block 1 ---');
myCoin.minePendingTransactions("Miner1");

// Create more transactions
myCoin.addTransaction(new Transaction("Charlie", "David", 10));
myCoin.addTransaction(new Transaction("Alice", "David", 5));

// Mine block 2
console.log('\n--- Mining Block 2 ---');
myCoin.minePendingTransactions("Miner2");

// Mine several blocks to trigger difficulty adjustment
for (let i = 0; i < 6; i++) {
    console.log(`\n--- Mining Block ${i + 3} ---`);
    myCoin.addTransaction(new Transaction(`User${i}`, `User${i+1}`, 1));
    myCoin.minePendingTransactions(`Miner${i+3}`);
}

// Display final state
myCoin.display();

// Check balances
console.log('\n--- ACCOUNT BALANCES ---');
console.log(`Miner1: ${myCoin.getBalanceOfAddress("Miner1")} BTC`);
console.log(`Miner2: ${myCoin.getBalanceOfAddress("Miner2")} BTC`);
console.log(`Alice: ${myCoin.getBalanceOfAddress("Alice")} BTC`);
console.log(`Bob: ${myCoin.getBalanceOfAddress("Bob")} BTC`);
console.log(`Charlie: ${myCoin.getBalanceOfAddress("Charlie")} BTC`);

// Demonstrate tampering detection
console.log('\n--- SECURITY TEST ---');
console.log('Tampering with Block 1...');
myCoin.chain[1].transactions[0].amount = 1000; // Try to change transaction

console.log(`Chain Valid after tampering: ${myCoin.isChainValid()}`);
console.log('❌ Tampering detected! Proof of Work prevents modification.');

/*
COMPLETE EXPLANATION:

1. TRANSACTION CLASS:
   - Represents a transfer of value
   - Contains from, to, amount, timestamp
   - Has its own hash for integrity

2. BLOCK CLASS:
   - Contains multiple transactions
   - Has previousHash linking to chain
   - Implements mineBlock() with PoW
   - Difficulty determines leading zeros required
   - Nonce incremented until valid hash found

3. BLOCKCHAIN CLASS:
   - Manages chain of blocks
   - Handles difficulty adjustment algorithm
   - Manages pending transaction pool
   - Validates proof of work on each block
   - Calculates balances by scanning chain

4. DIFFICULTY ADJUSTMENT:
   - Every 5 blocks, measures time taken
   - If too fast (<50% of expected), increases difficulty
   - If too slow (>200% of expected), decreases difficulty
   - Maintains consistent block time (3 seconds for demo)

5. SECURITY FEATURES:
   - Changing any transaction invalidates block hash
   - Invalid hash breaks chain linkage
   - isChainValid() detects any tampering
   - PoW makes recalculating all hashes prohibitively expensive

This is a complete, working implementation demonstrating all
key concepts of Proof of Work consensus.
*/
```

---

## **Chapter Summary**

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 5 SUMMARY                            │
│                                                                 │
│  1. PROOF OF WORK MECHANICS                                     │
│     • Miners compete to find nonce producing hash < target      │
│     • Difficulty adjusts to maintain consistent block times     │
│     • More leading zeros = exponentially harder                │
│                                                                 │
│  2. DIFFICULTY ADJUSTMENT                                       │
│     • Bitcoin adjusts every 2016 blocks (~2 weeks)             │
│     • Formula: New Difficulty = Old × (Actual Time/Target)      │
│     • Bounded adjustments (0.25x to 4x) prevent manipulation     │
│                                                                 │
│  3. SECURITY MODEL                                              │
│     • 51% attack: Control majority hash power to rewrite history │
│     • Economic cost makes attacks expensive                      │
│     • Energy consumption purchases security                      │
│                                                                 │
│  4. MINING EVOLUTION                                            │
│     • CPU (2009): General purpose, obsolete                    │
│     • GPU (2010): Parallel processing, 100x faster             │
│     • ASIC (2013): Specialized chips, 100,000x+ faster         │
│                                                                 │
│  5. MINING POOLS                                                │
│     • Combine hash power to reduce variance                    │
│     • Reward distribution: PPS, PPLNS, PROP                    │
│     • Centralization risk (few large pools)                    │
│                                                                 │
│  KEY TAKEAWAY:                                                  │
│  PoW achieves consensus through computational competition.        │
│  The cost of energy creates economic security.                   │
│  Difficulty ensures consistent block production despite          │
│  changing network power.                                        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Key Concepts Checklist**

Before proceeding, ensure you understand:

- [ ] How the nonce is used to find a valid block hash
- [ ] Why more leading zeros means higher difficulty
- [ ] The difficulty adjustment formula and its purpose
- [ ] What a 51% attack is and why it's expensive
- [ ] The trade-offs between CPU, GPU, and ASIC mining
- [ ] How mining pools reduce variance for individual miners
- [ ] Why PoW is secure (economic cost of attack)

### **Practice Questions**

1. **Explain why finding a valid block hash is difficult but verifying it is easy. What property of hash functions makes this possible?**

2. **If Bitcoin blocks are being mined every 8 minutes on average, what will happen to the difficulty at the next adjustment? Why?**

3. **Calculate: If the current difficulty target requires 4 leading zeros in hex, approximately how many hashes must a miner try on average to find a valid block? (Hint: Probability = 1/16^4)**

4. **Why is a 51% attack theoretically possible but practically difficult? Consider both technical and economic factors.**

5. **Compare solo mining vs. pool mining. Why do most miners join pools despite the fees?**

6. **Explain why ASICs created centralization pressure in Bitcoin mining. Is this a fundamental flaw in PoW or a market evolution?**

---

## **Coming Up Next: Chapter 6**

**Proof of Stake (PoS)**

In the next chapter, we'll explore the consensus mechanism that powers Ethereum 2.0 and many modern blockchains. You'll learn:

- **Staking Mechanics**: How validators lock up capital to secure the network
- **Validator Selection**: How the protocol chooses who creates the next block
- **Slashing Conditions**: Economic penalties for malicious behavior
- **The Nothing at Stake Problem**: Why early PoS systems failed and how modern solutions work
- **Ethereum's Transition**: The Merge and how Ethereum moved from PoW to PoS

We'll build a working Proof of Stake simulation where validators stake tokens, propose blocks, and face penalties for misbehavior—demonstrating how economic incentives replace energy consumption.

